In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

from time import sleep
from random import randint

In [2]:
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
headers = {"Accept-Language": "en-US, en;q=0.5"}

In [4]:
pages = np.arange(1, 1001, 50)

In [5]:
for page in pages: 
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&start=" + str(page) + "&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    sleep(randint(2,10))
    for container in movie_div:
        name = container.h3.a.text
        titles.append(name)
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        runtime = container.p.find('span', class_='runtime') if container.p.find('span', class_='runtime') else ''
        time.append(runtime)
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else ''
        metascores.append(m_score)
        nv = container.find_all('span', attrs={'name': 'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else ''
        us_gross.append(grosses)
movies = pd.DataFrame({
'movie': titles,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'us_grossMillions': us_gross,
'timeMin': time
})

In [6]:
movies.head()

,movie,year,imdb,metascore,votes,us_grossMillions,timeMin
0,The Gentlemen,(2019),8.0,51,"73,793",,"<span class=""runtime"">113 min</span>"
1,Knives Out,(2019),8.0,82,"248,809",$165.36M,"<span class=""runtime"">131 min</span>"
2,1917,(2019),8.4,78,"255,368",$159.18M,"<span class=""runtime"">119 min</span>"
3,Parasite,(2019),8.6,96,"339,294",$53.37M,"<span class=""runtime"">132 min</span>"
4,Once Upon a Time... in Hollywood,(2019),7.7,83,"421,918",$142.50M,"<span class=""runtime"">161 min</span>"


In [7]:
movies['votes'] = movies['votes'].str.replace(',', '').astype(int)

movies.loc[:, 'year'] = movies['year'].str[-5:-1].astype(int)

movies['timeMin'] = movies['timeMin'].astype(str)
movies['timeMin'] = movies['timeMin'].str.extract('(\d+)').astype(int)

movies['metascore'] = movies['metascore'].str.extract('(\d+)')
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')

In [8]:
print(movies)

# to see the datatypes of your columns
print(movies.dtypes)

# to see where you're missing data and how much data is missing
print(movies.isnull().sum())

                                                 movie  year  imdb  metascore  \
0                                        The Gentlemen  2019   8.0       51.0   
1                                           Knives Out  2019   8.0       82.0   
2                                                 1917  2019   8.4       78.0   
3                                             Parasite  2019   8.6       96.0   
4                     Once Upon a Time... in Hollywood  2019   7.7       83.0   
5                                           Uncut Gems  2019   7.5       90.0   
6                                          Jojo Rabbit  2019   8.0       58.0   
7                                                Joker  2019   8.5       59.0   
8                                       Ford v Ferrari  2019   8.1       81.0   
9                                         Little Women  2019   8.0       91.0   
10                            The Shawshank Redemption  1994   9.3       80.0   
11                          